In [ ]:
!pip install transformers datasets evaluate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import evaluate

## ***Get Dataset - BigCodeBench***

In [ ]:
dataset = load_dataset("bigcode/bigcodebench", split="v0.1.0_hf[:10]")

## ***Initialize Models***

In [ ]:
model_names = {                               #TODO : Change Model to check improvement
    "CodeLlama": "codellama/CodeLlama-7b-hf",
}

tokenizers = {}
models = {}

In [ ]:
for name, model_id in model_names.items():
    tokenizers[name] = AutoTokenizer.from_pretrained(model_id)
    models[name] = AutoModelForCausalLM.from_pretrained(model_id)

## ***Feedback Model***

In [ ]:
def generate_feedback():
    # TODO : Implement funtion for Reinforcement
    pass

In [1]:
def generate_code(prompt, model, tokenizer, solution, max_length=1000):
    n = 5 #TODO : Set N based on your inplementation
    for _ in range(n):
      inputs = tokenizer(prompt, return_tensors="pt")
      output = model.generate(**inputs, max_length=max_length, temperature=0.7, top_p=0.9, do_sample=True)
      bleu_score = bleu_metric.compute(predictions=output, references=solution)
      prompt = generate_feedback() #TODO : Implement funtion partametrs for Reinforcement
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
generated_codes = {name: [] for name in model_names}

for example in dataset:
    print("Prompt:")
    prompt = example["complete_prompt"]
    solution = example["canonical_solution"]
    for model_name in model_names:
        generated_code = generate_code(prompt, models[model_name], tokenizers[model_name], solution)
        generated_codes[model_name].append(generated_code)

## ***Evaluation***

In [ ]:
# Evaluation Setup with BLEU (or CodeBLEU if available)
bleu_metric = evaluate.load("bleu")

# Prepare reference code for evaluation
references = [example["canonical_solution"] for example in dataset]

# Evaluate each model's generated code against the reference code
evaluation_scores = {}
for model_name, codes in generated_codes.items():
    bleu_score = bleu_metric.compute(predictions=codes, references=references)
    evaluation_scores[model_name] = bleu_score["bleu"]
    print(f"{model_name} BLEU Score:", bleu_score["bleu"])

# Print final evaluation summary
print("\n=== Evaluation Summary ===")
for model_name, score in evaluation_scores.items():
    print(f"{model_name} BLEU Score: {score:.4f}")